# clean the Chroma DB:
-  on terminal:
rm -rf ./data/chroma_db
- delete list from "processed_files.txt"

# First experiment: Llama3.2

In [ ]:
def query_ollama(model: str, prompt: str) -> str:
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode('utf-8'),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    return result.stdout.decode('utf-8')

In [ ]:
prompt = """
I had an idea for a new project. Also I need to buy cat food. 
Later I thought maybe I should send that message to Alex.

1. Group related sentences.
2. Label each group as: [Idea, Task, Message]
3. Reorder and rewrite each clearly.
"""

response = query_ollama("mistral", prompt)
print(response)

 Here's the organized and rewritten version of your text according to your instructions:

[Idea]
- I had an idea for a new project.

[Task]
- Later I thought maybe I should buy cat food.

[Message]
- [Idea: Sharing my idea about a new project]
- [Task: Suggesting that Alex might help with buying cat food]

Rewritten and Reordered version:

1. First, I'd like to share an idea I had for a new project - do you think you could help discuss it?
2. Later, I thought it would be good if we could buy some cat food together as I need to take care of my pet. Let me know what you think.




It didn't work exactly how I expected and it took too long to give me an answer. Prompt engineering refinement, but still, hard to test. Changing approach to classical NLP and using generative AI models only for rewriting. 

# word tokenize + get chunks

- Analysis: The [word_toneknize] splits the punctiation into separate tokens and creates this view that makes harder to compreehend the context. I researched other tokenizer and found a "de-tokenizer" [tree bank word] to join the tokens in a more natural way.

In [ ]:
### function to split documents with Word+Tokenize ###

def split_by_words(text, chunk_size=80, overlap=20):
    words = word_tokenize(text)
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        if chunk:
            chunks.append(' '.join(chunk))
    return chunks

# 2. Split only new files into chunks of words
if not new_files:
    print("No new files to process.")
else:
    all_chunks = []
    all_metadatas = []
    for fname in new_files:
        with open(os.path.join(folder_path, fname), 'r', encoding='utf-8') as f:
            text = f.read()
            display(f"Text file: {fname}", Markdown(f"```\n{text}\n```"))
        chunks = split_by_words(text)
        all_chunks.extend(chunks)
        all_metadatas.extend([{"source": fname}] * len(chunks))    

In [ ]:
### get the chunks from a specific file

def get_file_chunks(fname):
    results = vectorstore.get(where={"source": fname})
    file_chunks = results['documents']
    print(f"Found {len(file_chunks)} chunks from {fname}")
    for chunk in file_chunks:
        display(Markdown(f"```\n{chunk}\n```"))

file1_chunks = get_file_chunks("note-1-dt-r.txt")

Found 12 chunks from note-1-dt-r.txt


```
entropy = -p * log2 ( p ) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node . Gini impurity also similar but faster ? less bias ? not 100 % sure . trees go deep and then prone back ? no , prune . to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right
```

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right ? sklearn.tree.DecisionTreeClassifier ( max_depth=3 ) — yeah that ’ s what prof used . note : good to visualize trees but not for high dim data . lots of axis-aligned splits , hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum ( w² ) , shrinks weights , but all stay ≠ 0 . Lasso adds λ * sum ( |w| ) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board : from sklearn.linear_model import Lasso model = Lasso ( alpha=0.1 ) model.fit ( X_train , y_train ) ElasticNet = mix of both ? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting .
```

...

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right ? sklearn.tree.DecisionTreeClassifier ( max_depth=3 ) — yeah that ’ s what prof used . note : good to visualize trees but not for high dim data . lots of axis-aligned splits , hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum ( w² ) , shrinks weights , but all stay ≠ 0 . Lasso adds λ * sum ( |w| ) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board : from sklearn.linear_model import Lasso model = Lasso ( alpha=0.1 ) model.fit ( X_train , y_train ) ElasticNet = mix of both ? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting .
```

```
good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting . btw they said don ’ t scale trees but do scale for lasso etc . bcz regularization depends on magnitude . std scaling or minmax okay . might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV decision boundary of tree is step-like , not smooth like linear models . ok Q : why trees overfit more than lasso ? more flexible
```

```
step-like , not smooth like linear models . ok Q : why trees overfit more than lasso ? more flexible model class I think ?
```

### 2. Document segmentation (text split)
Split the document into chunks of 80 words, which is enough to capture the semantic context. Sentences and words would miss the context. Paragraphs are not consistent with unstructured documents.

In [ ]:
### function to split document with TreebankWordDetokenizer ###

def split_by_words_de(text, chunk_size=80, overlap=20):
    words = word_tokenize(text)
    detok = TreebankWordDetokenizer()                       # detokenizer to join the tokens in a more natural way
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        if chunk:
            chunks.append(detok.detokenize(chunk))
    return chunks

In [ ]:
# 2. Split document into chunks of words

if not new_files:
    print("No new files to process.")
else:
    all_chunks = []
    all_metadatas = []
    for fname in new_files:
        with open(os.path.join(folder_path, fname), 'r', encoding='utf-8') as f:
            text = f.read()
    
        chunks = split_by_words_de(text)                                    #split in chunks
        display(f"Original file: {fname}", Markdown(f"```\n{text}\n```"))   #display original file(s)
        print(f"{fname} = {len(chunks)} chunks")                            #display number of chunks per file
        for i, chunk in enumerate(chunks, 1):
            display(Markdown(f"**Chunk {i}:**\n{chunk})\n"))
        all_chunks.extend(chunks)
        all_metadatas.extend([{"source": fname}] * len(chunks))   

'Original file: note-3-dt-r-ul.txt'

```
lecture today was fast af. started with trees. entropy vs gini impurity — diff metrics to decide best split. both OK. CART = binary tree = each node has 2 splits. sklearn uses this. code: from sklearn.tree import DecisionTreeClassifier model = DecisionTreeClassifier(criterion='entropy', max_depth=4)
trees prone to overfit — esp if depth unbounded. pruning = way to fix. early stopping or post-prune. bagging helps = ensemble. RandomForest = multiple trees on bootstrapped samples + rand subset of features per split. reduces variance.
trees = interpretability good, but unstable to data changes.
then prof jumped into regression. regularization = shrink model capacity. Ridge = L2 norm = λ * Σ(w²). Lasso = L1 norm = λ * Σ|w|. Ridge keeps all weights ≠ 0, Lasso can zero out → sparse. Lasso good for feature selection. ElasticNet = mix of both — good if features correlated
code ex:
from sklearn.linear_model import ElasticNet model = ElasticNet(alpha=0.1, l1_ratio=0.5) model.fit(X, y)
important: scale features before fitting regularized models — otherwise magnitudes skew the penalty. StandardScaler or RobustScaler if outliers.
tune α via cross-val — use GridSearchCV or RandomizedSearchCV.
metrics: RMSE, R². underfit vs overfit — regularization helps balance bias/var tradeoff.
last part was unsupervised learning. clustering w/o labels. k-means = most used. init centers, assign pts, recalc, repeat. problem: sensitive to init. use k-means++. elbow method not always clear. silhouette score better maybe.
PCA + k-means often combined for vis. t-SNE only for viz — not for modeling. clusters in t-SNE are sometimes fake.
DBSCAN = cluster via density. can detect noise. great for shape-agnostic clusters. params hard to tune tho. hierarchical = dendrograms. use ‘ward’ linkage. but slow w/ big data.
general note: sklearn models consistent API — fit / predict / score.
also: why trees don’t need scaling? bcz splits based on order not value. contrast w/ reg models.
each algo has tradeoffs — no free lunch! choose based on data, task, interpretability needs.
```

note-3-dt-r-ul.txt = 7 chunks


**Chunk 1:**
lecture today was fast af . started with trees . entropy vs gini impurity — diff metrics to decide best split . both OK. CART = binary tree = each node has 2 splits . sklearn uses this . code: from sklearn.tree import DecisionTreeClassifier model = DecisionTreeClassifier (criterion='entropy', max_depth=4) trees prone to overfit — esp if depth unbounded . pruning = way to fix . early stopping or post-prune . bagging helps = ensemble.)


**Chunk 2:**
if depth unbounded . pruning = way to fix . early stopping or post-prune . bagging helps = ensemble . RandomForest = multiple trees on bootstrapped samples + rand subset of features per split . reduces variance . trees = interpretability good, but unstable to data changes . then prof jumped into regression . regularization = shrink model capacity . Ridge = L2 norm = λ * Σ (w²). Lasso = L1 norm = λ *)


...

**Chunk 4:**
(X) hierarchical clustering = agglomerative or divisive — we focus on bottom-up (agglomerative). start w all pts as indiv cluster, merge closest pairs step by step . dendrogram = tree of merges . can "cut" tree at diff levels = diff num clusters . linkage: single, complete, avg . sklearn has AgglomerativeClustering . before clustering, can reduce dim (PCA) for speed + viz . tSNE/UMAP for)


**Chunk 5:**
sklearn has AgglomerativeClustering . before clustering, can reduce dim (PCA) for speed + viz . tSNE/UMAP for 2D plot = better for human eye but not for modeling . PCA = linear, tSNE = non-linear . tSNE distorts structure globally . good for pattern discovery . spectral clustering = build similarity graph → Laplacian → eigenvectors → k-means in lower-dim eigenspace . nice when structure is graphy, not spherical . most clustering algos rely on)


**Chunk 6:**
→ k-means in lower-dim eigenspace . nice when structure is graphy, not spherical . most clustering algos rely on distance metric — Euclidean default . alt: cosine sim (for text), manhattan, etc . eval: hard bcz no true label . silhouette best for most . DB index too . compare within/between cluster distance . can also visualize clusters to judge quality . problem: k-means assumes spherical clusters, equal size . not)


**Chunk 7:**
. can also visualize clusters to judge quality . problem: k-means assumes spherical clusters, equal size . not true for real-world . if data has diff density or shapes → fails . open Q: how to know if clusters mean anything in real world? scaling is essential — StandardScaler or MinMaxScaler from sklearn . pipeline ex: from sklearn.pipeline import make_pipeline pipe = make_pipeline (StandardScaler (), KMeans (n_clusters=3)) pipe.fit ()


**Chunk 8:**
: from sklearn.pipeline import make_pipeline pipe = make_pipeline (StandardScaler (), KMeans (n_clusters=3)) pipe.fit (X))


### 3. Embedding new chunks from new files

In [ ]:
# 3.1. Embed new chunks
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')


# 3.2. Add embeddings to Chroma DB
persist_dir = "./data/chroma_db"

if os.path.exists(persist_dir):
    print(f"Loading existing Chroma DB from: {persist_dir}")
    vectorstore = Chroma(
        persist_directory=persist_dir,
        embedding=embeddings
    )
    vectorstore.add_texts(texts=all_chunks, metadatas=all_metadatas)
else:
    print(f"Creating new Chroma DB in: {persist_dir}")
    vectorstore = Chroma.from_texts(
        texts=all_chunks, 
        metadatas=all_metadatas, 
        embedding=embeddings, 
        persist_directory=persist_dir
    )
vectorstore.persist()


# 3.3. Update processed files log
with open(log_path, "a") as f:
    for fname in new_files:
        f.write(fname + "\n")
print(f"Processed and added {len(new_files)} new files to Chroma DB.")

/var/folders/_5/9xl167095gn7qzk16djnlv300000gp/T/ipykernel_34806/884519275.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')


Creating new Chroma DB in: ./data/chroma_db
Processed and added 3 new files to Chroma DB.


/var/folders/_5/9xl167095gn7qzk16djnlv300000gp/T/ipykernel_34806/884519275.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


### 4. Clusterize by semantic similarity
1 exprimente: HDBSCAN - Hierarchical Density-Based Spatial Clustering of Applications with Noise -> not good. moved to other file
2 experiment: kmeans


In [ ]:
### function to retrieve the chunks from a specific file

def get_file_chunks(fname):
    results = vectorstore.get(where={"source": fname})
    file_chunks = results['documents']
    print(f"Found {len(file_chunks)} chunks from {fname}")
    for chunk in file_chunks:
        display(Markdown(f"```\n{chunk}\n```"))             # display the contents of each chunk for conference
    return file_chunks                                      # return the list for further use

In [ ]:
# 4.1. Get embeddings chunks from File 1

file1_chunks = get_file_chunks("note-1-dt-r.txt")
file1_embeddings = embeddings.embed_documents(file1_chunks)

Found 6 chunks from note-1-dt-r.txt


```
entropy = -p * log2 (p) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node . Gini impurity also similar but faster? less bias? not 100% sure . trees go deep and then prone back? no, prune . to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right
```

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right? sklearn.tree.DecisionTreeClassifier (max_depth=3) — yeah that ’ s what prof used . note: good to visualize trees but not for high dim data . lots of axis-aligned splits, hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum (w²), shrinks weights, but all stay ≠ 0 . Lasso adds λ * sum (|w|) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board: from sklearn.linear_model import Lasso model = Lasso (alpha=0.1) model.fit (X_train, y_train) ElasticNet = mix of both? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting.
```

```
good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting . btw they said don ’ t scale trees but do scale for lasso etc . bcz regularization depends on magnitude . std scaling or minmax okay . might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV decision boundary of tree is step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible
```

```
step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible model class I think?
```

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 2  # or 1, 2, 3 depending on your expectation
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(file1_embeddings)
print("KMeans cluster labels:", labels)

KMeans cluster labels: [0 0 0 1 1 1]


In [ ]:
### Function to display chunks in clusters  ###

for cluster_id in set(labels):
    print(f"\n--- Cluster {cluster_id} ---")
    for i, label in enumerate(labels):
        if label == cluster_id:
            print(f"Chunk {i+1}:")
            #print(file1_chunks[i]) 
            display(Markdown(file1_chunks[i]))
            print()


--- Cluster 0 ---
Chunk 1:


entropy = -p * log2 (p) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node . Gini impurity also similar but faster? less bias? not 100% sure . trees go deep and then prone back? no, prune . to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right


Chunk 2:


. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right? sklearn.tree.DecisionTreeClassifier (max_depth=3) — yeah that ’ s what prof used . note: good to visualize trees but not for high dim data . lots of axis-aligned splits, hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that


Chunk 3:


...

— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board: from sklearn.linear_model import Lasso model = Lasso (alpha=0.1) model.fit (X_train, y_train) ElasticNet = mix of both? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting.


Chunk 5:


good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting . btw they said don ’ t scale trees but do scale for lasso etc . bcz regularization depends on magnitude . std scaling or minmax okay . might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV decision boundary of tree is step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible


Chunk 6:


step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible model class I think?

chat gpt splitting: 
1. entropy = -p * log2(p) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node. Gini impurity also similar but faster? less bias? not 100% sure. trees go deep and then prone back? no, prune. to prevent overfitting. training error low but generalization bad. CART uses binary splits – only yes/no right? sklearn.tree.DecisionTreeClassifier(max_depth=3) — yeah that’s what prof used.
note: good to visualize trees but not for high dim data. lots of axis-aligned splits, hard to interpret when too many features. oh and trees are unstable — small data change = big model change. they said bagging can help that — RandomForest.
decision boundary of tree is step-like, not smooth like linear models.
ok Q: why trees overfit more than lasso? more flexible model class I think?
btw they said don’t scale trees

2. wait then they jumped to regularization — lasso vs ridge. ridge adds λ * sum(w²), shrinks weights, but all stay ≠ 0. Lasso adds λ * sum(|w|) — forces some to zero. ohhh good for feature selection. balance bias-variance tradeoff. λ too big = underfit. low λ = overfit. prof wrote this on board: from sklearn.linear_model import Lasso model = Lasso(alpha=0.1) model.fit(X_train, y_train)
ElasticNet = mix of both? ratio param controls mix. good when multicollinearity or many small coeffs. visualize loss function — lasso diamond corners cause zeros. interesting.
btw they said […] do scale for lasso etc. bcz regularization depends on magnitude. std scaling or minmax okay.
might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(file1_embeddings)  # embeddings: list of vectors
print(similarity_matrix)

# Interpretation:
# 1.0: Vectors are identical in direction (maximum similarity).
# 0.0: Vectors are orthogonal (no similarity).
# -1.0: Vectors are opposite in direction (rare in NLP embeddings).

[[1.         0.54851236 0.37322338 0.11788947 0.25886513 0.30239811]
 [0.54851236 1.         0.44631097 0.33833166 0.44378826 0.47022212]
 [0.37322338 0.44631097 1.         0.49479337 0.6547976  0.52447321]
 [0.11788947 0.33833166 0.49479337 1.         0.59648798 0.45439273]
 [0.25886513 0.44378826 0.6547976  0.59648798 1.         0.62814475]
 [0.30239811 0.47022212 0.52447321 0.45439273 0.62814475 1.        ]]


# experiment HDBSCAN:
the results were -1, which means that everything was considered noise. 

In [ ]:
### function to retrieve the chunks from a specific file

def get_file_chunks(fname):
    results = vectorstore.get(where={"source": fname})
    file_chunks = results['documents']
    print(f"Found {len(file_chunks)} chunks from {fname}")
    for chunk in file_chunks:
        display(Markdown(f"```\n{chunk}\n```"))             # <-- display the contents of each chunk for conference
    return file_chunks                                      # <-- return the list for further use
 

In [ ]:
# 4.1. Get embeddings chunks from File 1

file1_chunks = get_file_chunks("note-1-dt-r.txt")
file1_embeddings = embeddings.embed_documents(file1_chunks)


# 4.2. Initialize and fit HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=2)
labels = cluster.fit_predict(file1_embeddings)

print("Cluster labels for File 1:", labels)

Found 6 chunks from note-1-dt-r.txt


```
entropy = -p * log2 (p) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node . Gini impurity also similar but faster? less bias? not 100% sure . trees go deep and then prone back? no, prune . to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right
```

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right? sklearn.tree.DecisionTreeClassifier (max_depth=3) — yeah that ’ s what prof used . note: good to visualize trees but not for high dim data . lots of axis-aligned splits, hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum (w²), shrinks weights, but all stay ≠ 0 . Lasso adds λ * sum (|w|) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board: from sklearn.linear_model import Lasso model = Lasso (alpha=0.1) model.fit (X_train, y_train) ElasticNet = mix of both? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting.
```

```
good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting . btw they said don ’ t scale trees but do scale for lasso etc . bcz regularization depends on magnitude . std scaling or minmax okay . might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV decision boundary of tree is step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible
```

```
step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible model class I think?
```

Cluster labels for File 1: [-1 -1 -1 -1 -1 -1]


In [ ]:
file2_chunks = get_file_chunks("note-2-ul.txt")
file2_embeddings = embeddings.embed_documents(file1_chunks)


# 4.2. Initialize and fit HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=2)
labels = cluster.fit_predict(file2_embeddings)

print("Cluster labels for File 2:", labels)

Found 8 chunks from note-2-ul.txt


```
unsup = no labels . kmeans = simplest one but still used a lot . init k centroids randomly (k-means++ better), assign pts, recalc centroids, repeat . converge when no pt changes . but result depends on init + scale . scale important! feature w bigger range dominates dist calc — always standardize first . elbow method = plot inertia vs k — look for bend, but not always obvious . inertia =
```

```
. elbow method = plot inertia vs k — look for bend, but not always obvious . inertia = sum of dist² to centroid . alt metric = silhouette score — between -1 and 1. close to 1 = well-clustered . clustering ≠ classification . labels are not known . use cases: market segmentation, gene expr clustering, anomaly detection (esp dbscan). DBSCAN better for weird shapes, dense clusters — uses eps +
```

```
, anomaly detection (esp dbscan). DBSCAN better for weird shapes, dense clusters — uses eps + min_samples . tricky to tune tho . forms clusters based on density, noisy pts marked as outliers (label -1). sklearn DBSCAN ex: from sklearn.cluster import DBSCAN model = DBSCAN (eps=0.5, min_samples=5) model.fit (X) hierarchical clustering = agglomerative or divisive — we focus on bottom-up (agglomerative). start w
```

```
(X) hierarchical clustering = agglomerative or divisive — we focus on bottom-up (agglomerative). start w all pts as indiv cluster, merge closest pairs step by step . dendrogram = tree of merges . can "cut" tree at diff levels = diff num clusters . linkage: single, complete, avg . sklearn has AgglomerativeClustering . before clustering, can reduce dim (PCA) for speed + viz . tSNE/UMAP for
```

```
sklearn has AgglomerativeClustering . before clustering, can reduce dim (PCA) for speed + viz . tSNE/UMAP for 2D plot = better for human eye but not for modeling . PCA = linear, tSNE = non-linear . tSNE distorts structure globally . good for pattern discovery . spectral clustering = build similarity graph → Laplacian → eigenvectors → k-means in lower-dim eigenspace . nice when structure is graphy, not spherical . most clustering algos rely on
```

```
→ k-means in lower-dim eigenspace . nice when structure is graphy, not spherical . most clustering algos rely on distance metric — Euclidean default . alt: cosine sim (for text), manhattan, etc . eval: hard bcz no true label . silhouette best for most . DB index too . compare within/between cluster distance . can also visualize clusters to judge quality . problem: k-means assumes spherical clusters, equal size . not
```

```
. can also visualize clusters to judge quality . problem: k-means assumes spherical clusters, equal size . not true for real-world . if data has diff density or shapes → fails . open Q: how to know if clusters mean anything in real world? scaling is essential — StandardScaler or MinMaxScaler from sklearn . pipeline ex: from sklearn.pipeline import make_pipeline pipe = make_pipeline (StandardScaler (), KMeans (n_clusters=3)) pipe.fit (
```

```
: from sklearn.pipeline import make_pipeline pipe = make_pipeline (StandardScaler (), KMeans (n_clusters=3)) pipe.fit (X)
```

Cluster labels for File 2: [-1 -1 -1 -1 -1 -1]


In [ ]:
file3_chunks = get_file_chunks("note-3-dt-r-ul.txt")
file3_embeddings = embeddings.embed_documents(file3_chunks)


# 4.2. Initialize and fit HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=2)
labels = cluster.fit_predict(file3_embeddings)

print("Cluster labels for File 1:", labels)

Found 7 chunks from note-3-dt-r-ul.txt


```
lecture today was fast af . started with trees . entropy vs gini impurity — diff metrics to decide best split . both OK. CART = binary tree = each node has 2 splits . sklearn uses this . code: from sklearn.tree import DecisionTreeClassifier model = DecisionTreeClassifier (criterion='entropy', max_depth=4) trees prone to overfit — esp if depth unbounded . pruning = way to fix . early stopping or post-prune . bagging helps = ensemble.
```

```
if depth unbounded . pruning = way to fix . early stopping or post-prune . bagging helps = ensemble . RandomForest = multiple trees on bootstrapped samples + rand subset of features per split . reduces variance . trees = interpretability good, but unstable to data changes . then prof jumped into regression . regularization = shrink model capacity . Ridge = L2 norm = λ * Σ (w²). Lasso = L1 norm = λ *
```

```
. Ridge = L2 norm = λ * Σ (w²). Lasso = L1 norm = λ * Σ|w| . Ridge keeps all weights ≠ 0, Lasso can zero out → sparse . Lasso good for feature selection . ElasticNet = mix of both — good if features correlated code ex: from sklearn.linear_model import ElasticNet model = ElasticNet (alpha=0.1, l1_ratio=0.5) model.fit (X, y) important: scale features before fitting regularized
```

```
= ElasticNet (alpha=0.1, l1_ratio=0.5) model.fit (X, y) important: scale features before fitting regularized models — otherwise magnitudes skew the penalty . StandardScaler or RobustScaler if outliers . tune α via cross-val — use GridSearchCV or RandomizedSearchCV . metrics: RMSE, R² . underfit vs overfit — regularization helps balance bias/var tradeoff . last part was unsupervised learning . clustering w/o labels . k-means = most used . init centers, assign pts
```

```
last part was unsupervised learning . clustering w/o labels . k-means = most used . init centers, assign pts, recalc, repeat . problem: sensitive to init . use k-means++ . elbow method not always clear . silhouette score better maybe . PCA + k-means often combined for vis . t-SNE only for viz — not for modeling . clusters in t-SNE are sometimes fake . DBSCAN = cluster via density . can detect noise . great
```

```
modeling . clusters in t-SNE are sometimes fake . DBSCAN = cluster via density . can detect noise . great for shape-agnostic clusters . params hard to tune tho . hierarchical = dendrograms . use ‘ ward ’ linkage . but slow w/ big data . general note: sklearn models consistent API — fit / predict / score . also: why trees don ’ t need scaling? bcz splits based on order not value . contrast w/
```

```
also: why trees don ’ t need scaling? bcz splits based on order not value . contrast w/ reg models . each algo has tradeoffs — no free lunch! choose based on data, task, interpretability needs.
```

Cluster labels for File 1: [-1 -1 -1 -1 -1 -1 -1]


### old class clusterizer

In [ ]:
class ChunkClusterizer:
    def __init__(self, persist_dir, embeddings):
        self.persist_dir = persist_dir
        self.embeddings = embeddings
        self.chunks = []
        self.labels = []
        self.vectorstore = Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embeddings
        )
        
    def get_file_chunks(self, fname):
        results = self.vectorstore.get(where={"source": fname})
        self.chunks = results['documents']
        print(f"Found {len(self.chunks)} chunks in {fname}")
        for chunk in self.chunks:
            display(Markdown(f"```\n{chunk}\n```"))
        return self.chunks

    def cluster_chunks(self, min_cluster_size=2, min_samples=2):
        print("Clustering chunks with HDBSCAN...")
        X = self.embeddings.embed_documents(self.chunks)
        clusterizer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
        self.labels = clusterizer.fit_predict(X)

        for cluster_id in sorted(set(self.labels)):
            print(f"\n--- Cluster {cluster_id} ---")
            for i, label in enumerate(self.labels):
                if label == cluster_id:
                    print(f"Chunk {i+1}:\n{self.chunks[i][:200]}{'...' if len(self.chunks[i]) > 200 else ''}\n")
        return self.labels

In [ ]:
class ChunkClusterizer:
    def __init__(self, chunks, chunk_embeddings):
        self.chunks = chunks
        self.embeddings = np.array(chunk_embeddings) if chunk_embeddings else None
        self.labels = []

    def cluster_chunks(self, min_cluster_size=2, min_samples=1):
        print("Clustering chunks with HDBSCAN...")
        
        if not self.chunks:
            print("No chunks to cluster.")
            return []
            
        if len(self.chunks) < min_cluster_size:
            print(f"Not enough chunks to cluster. Need at least {min_cluster_size}, but have {len(self.chunks)}.")
            self.labels = [-1] * len(self.chunks)
            return self.labels

        # Fix: Check numpy array properly
        if self.embeddings is None or self.embeddings.size == 0:
            print("No embeddings available for clustering.")
            self.labels = [-1] * len(self.chunks)
            return self.labels

        try:
            # Ensure embeddings is a 2D array
            if self.embeddings.ndim == 1:
                self.embeddings = self.embeddings.reshape(1, -1)
            
            print(f"Clustering {len(self.chunks)} chunks with embeddings shape: {self.embeddings.shape}")
            
            clusterizer = hdbscan.HDBSCAN(
                min_cluster_size=min_cluster_size, 
                min_samples=min_samples,
                metric='euclidean'
            )
            self.labels = clusterizer.fit_predict(self.embeddings)

            num_clusters = len(set(self.labels)) - (1 if -1 in self.labels else 0)
            print(f"\nFound {num_clusters} clusters in the document.")
            print(f"Cluster distribution: {dict(zip(*np.unique(self.labels, return_counts=True)))}")
            
            for cluster_id in sorted(set(self.labels)):
                cluster_name = "Noise (ungrouped chunks)" if cluster_id == -1 else f"Cluster {cluster_id}"
                chunk_count = sum(1 for label in self.labels if label == cluster_id)
                print(f"\n--- {cluster_name} ({chunk_count} chunks) ---")
                
                for i, label in enumerate(self.labels):
                    if label == cluster_id:
                        preview = self.chunks[i][:200] + ('...' if len(self.chunks[i]) > 200 else '')
                        print(f"Chunk {i+1}: {preview}\n")
            
            return self.labels
            
        except Exception as e:
            print(f"Error during clustering: {e}")
            self.labels = [-1] * len(self.chunks)
            return self.labels

# code backup

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import nltk
from transformers import AutoTokenizer
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
from IPython.display import Markdown, display
import hdbscan
import numpy as np

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/work/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 1. Input unstructured document
Incremental version. It will process only new files.

In [ ]:
# Config paths

FOLDER_PATH = "./data/mock_notes/"          #files
LOG_PATH = "./data/processed_files.txt"     #list of processed files
PERSIST_DIR = "./data/chroma_db"            #embeddings database (store vectorized chunks)

# Create directories if they don't exist
os.makedirs(FOLDER_PATH, exist_ok=True)
os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)
os.makedirs(PERSIST_DIR, exist_ok=True)

#### 1.2. Check if the input is already processed or is a new file

In [ ]:
def get_processed_files():
    if os.path.exists(LOG_PATH):
        with open(LOG_PATH, "r") as f:
            return set(line.strip() for line in f)
    return set()


def get_new_files():
    if not os.path.exists(FOLDER_PATH):
        print(f"Folder {FOLDER_PATH} does not exist.")
        return []
    all_files = [f for f in os.listdir(FOLDER_PATH) if f.endswith(".txt")]
    processed_files = get_processed_files()
    return [f for f in all_files if f not in processed_files]

### 2. Split document into chunks of tokens

In [ ]:
# Tokenizer & chunk helpers
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def count_tokens(text):
    return len(tokenizer.tokenize(text))

def is_code_like(sentence):
    code_keywords = ["from ", "import ", "def ", "return ", "class ", "for ", "if ", "else", "while ", "try:", "except", "print("]
    code_symbols = ["=", "()", "[]", "{}", "->", "::"]
    sentence_lower = sentence.strip().lower()
    return (
        any(sentence_lower.startswith(k) for k in code_keywords)
        or any(sym in sentence for sym in code_symbols)
    )


# Generate chunks
def chunk_text(text, source_name, min_tokens=40, max_tokens=100, inspect=False):
    raw_blocks = re.split(r"\n\s*\n", text.strip())
    chunks, current_chunk, current_tokens = [], [], 0
    metadata = []

    for block in raw_blocks:
        sentences = nltk.sent_tokenize(block.strip())

        for sentence in sentences:
            sentence = sentence.strip()
            tokens = count_tokens(sentence)

            if is_code_like(sentence):
                if current_chunk:
                    chunks.append(" ".join(current_chunk))
                    metadata.append({"source": source_name})
                    current_chunk, current_tokens = [], 0
                chunks.append(sentence)
                metadata.append({"source": source_name})
                continue

            if any(sentence.lower().startswith(w) for w in ["btw", "wait", "oh", "then", "also", "now", "next"]):
                if current_chunk:
                    chunks.append(" ".join(current_chunk))
                    metadata.append({"source": source_name})
                    current_chunk, current_tokens = [], 0

            if current_tokens + tokens > max_tokens and current_tokens >= min_tokens:
                chunks.append(" ".join(current_chunk))
                metadata.append({"source": source_name})
                current_chunk, current_tokens = [], 0

            current_chunk.append(sentence)
            current_tokens += tokens

        if current_chunk:
            chunks.append(" ".join(current_chunk))
            metadata.append({"source": source_name})
            current_chunk, current_tokens = [], 0

    if inspect:
        for i, chunk in enumerate(chunks):
            print(f"\nChunk {i+1} ({count_tokens(chunk)} tokens):\n{chunk}")

    return chunks, metadata

### 3. Embed all chunks

#### 3.1. Initialize sentence-transformer model

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#### 3.2. Save embeddings into Chroma DB

In [ ]:
class VectorStoreManager:
    def __init__(self, persist_dir, embeddings):
        self.persist_dir = persist_dir
        self.embeddings = embeddings
        self.vectorstore = self._load_existing()

    def _load_existing(self):
        """Attempts to load an existing Chroma DB from the persist directory."""
        try:
            if os.path.exists(self.persist_dir) and os.listdir(self.persist_dir):
                print(f"Loading existing Chroma DB from {self.persist_dir}")
                return Chroma(
                    persist_directory=self.persist_dir,
                    embedding_function=self.embeddings,
                    collection_name="default"  # Add this line
                )
        except Exception as e:
            print(f"Error loading existing Chroma DB: {e}")
            print("Will create a new one.")
        
        print("\n-> No existing Chroma DB found. A new one will be created when chunks are added.")
        return None

    def add_chunks(self, chunks, metadata):
        """Adds chunks to the vector store, creating it if it doesn't exist."""
        if not chunks:
            print("No chunks to add.")
            return

        if not self.vectorstore:
            print(f"\n-> Creating new Chroma DB at {self.persist_dir}")
            self.vectorstore = Chroma.from_texts(
                texts=chunks,
                metadatas=metadata,
                embedding=self.embeddings,  # Changed from embedding_function
                persist_directory=self.persist_dir,
                collection_name="default"   # Add this line
            )
        else:
            self.vectorstore.add_texts(texts=chunks, metadatas=metadata)

    def get_chunks_by_file(self, fname):
        """Retrieves all chunks associated with a specific source file."""
        if not self.vectorstore:
            print("Vectorstore is not initialized. Cannot retrieve chunks.")
            return []
        results = self.vectorstore.get(where={"source": fname})
        return results.get('documents', []) # Safely get documents

    def get_all_chunks(self):
        """Retrieves all chunks from the vector store."""
        if not self.vectorstore:
            print("Vectorstore is not initialized. Cannot retrieve chunks.")
            return []
        results = self.vectorstore.get(include=["documents"])
        return results.get('documents', [])
    
    def get_chunks_with_embeddings(self, fname):
        """Retrieves chunks and their embeddings for a specific file."""
        if not self.vectorstore:
            print("Vectorstore is not initialized. Cannot retrieve chunks.")
            return [], []
        
        try:
            results = self.vectorstore.get(
                where={"source": fname},
                include=["documents", "embeddings"]
            )
            chunks = results.get('documents', [])
            embeddings_list = results.get('embeddings', [])
            
            if not chunks:
                print(f"No chunks found for file: {fname}")
                return [], []
            
            if embeddings_list is None or len(embeddings_list) == 0:
                print(f"No embeddings found for file: {fname}. Computing embeddings...")
                # Fallback: compute embeddings manually if not stored
                embeddings_list = [self.embeddings.embed_query(chunk) for chunk in chunks]
            
            return chunks, embeddings_list
        except Exception as e:
            print(f"Error retrieving chunks with embeddings for file {fname}: {e}")
            return [], []

### 4. Cluster the embeddings by semantic similarity

In [ ]:
class ChunkClusterizer:
    def __init__(self, chunks, chunk_embeddings):
        self.chunks = chunks                                                        # Store text chunks for processing
        self.embeddings = np.array(chunk_embeddings) if chunk_embeddings else None  # Convert embeddings to numpy array for ML operations
        self.labels = []                                                            # Initialize empty list to store cluster labels

    def cluster_chunks(self, min_cluster_size=2, min_samples=1):
        print("Clustering chunks with HDBSCAN...")
        
        if not self.chunks:                                                    # Check if chunks list is empty
            print("No chunks to cluster.")
            return []
            
        if len(self.chunks) < min_cluster_size:                                # Check if we have enough chunks to form clusters
            print(f"Not enough chunks to cluster. Need at least {min_cluster_size}, but have {len(self.chunks)}.")
            self.labels = [-1] * len(self.chunks)                              # Assign all chunks to noise cluster (-1)
            return self.labels

        if self.embeddings is None or self.embeddings.size == 0:              # Check if embeddings exist and are not empty
            print("No embeddings available for clustering.")
            self.labels = [-1] * len(self.chunks)                             # Assign all chunks to noise cluster (-1)
            return self.labels

        try:
            if self.embeddings.ndim == 1:                                     # Check if embeddings are 1-dimensional
                self.embeddings = self.embeddings.reshape(1, -1)              # Reshape to 2D: (1, n_features)
            
            print(f"Clustering {len(self.chunks)} chunks with embeddings shape: {self.embeddings.shape}")
            
            clusterizer = hdbscan.HDBSCAN(                                     # Initialize HDBSCAN clustering algorithm
                min_cluster_size=min_cluster_size,                             # Minimum number of points in a cluster
                min_samples=min_samples,                                       # Minimum samples in neighborhood for core point
                metric='euclidean'                                             # Distance metric for clustering
            )
            self.labels = clusterizer.fit_predict(self.embeddings)             # Fit model and predict cluster labels

            num_clusters = len(set(self.labels)) - (1 if -1 in self.labels else 0)  # Count unique clusters (excluding noise)
            print(f"\nFound {num_clusters} clusters in the document.")
            print(f"Cluster distribution: {dict(zip(*np.unique(self.labels, return_counts=True)))}")  # Show cluster sizes
            
            for cluster_id in sorted(set(self.labels)):                       # Iterate through each unique cluster ID
                cluster_name = "Noise (ungrouped chunks)" if cluster_id == -1 else f"Cluster {cluster_id}"  # Name clusters
                chunk_count = sum(1 for label in self.labels if label == cluster_id)  # Count chunks in this cluster
                print(f"\n--- {cluster_name} ({chunk_count} chunks) ---")
                
                for i, label in enumerate(self.labels):                       # Iterate through all chunk labels
                    if label == cluster_id:                                   # Check if chunk belongs to current cluster
                        preview = self.chunks[i][:200] + ('...' if len(self.chunks[i]) > 200 else '')  # Create preview text
                        print(f"Chunk {i+1}: {preview}\n")                    # Print chunk preview
            
            return self.labels                                                # Return cluster labels
            
        except Exception as e:                                                # Handle any clustering errors
            print(f"Error during clustering: {e}")
            self.labels = [-1] * len(self.chunks)                             # Assign all chunks to noise on error
            return self.labels

### Entry point to run steps 1 to 4

In [ ]:
def process_file(fname):
    if fname in get_processed_files():
        print(f"Skipping {fname} (already processed).")
        return

    print(f"\nProcessing new file: {fname}")
    # 1. Read text
    with open(os.path.join(FOLDER_PATH, fname), "r", encoding="utf-8") as f:
        text = f.read()

    # 2. Create chunks
    chunks, metadata = chunk_text(text, fname, inspect=True)

    # 3. Initialize vector manager
    vector_manager = VectorStoreManager(PERSIST_DIR, embeddings)
    vector_manager.add_chunks(chunks, metadata)

    # 4. Mark file as processed 
    with open(LOG_PATH, "a") as log:
        log.write(fname + "\n")

    print(f"\nAdded {len(chunks)} chunks from {fname} to Chroma DB.")

    # 5. Cluster chunks from new file
    print(f"\n--- Starting Clustering Process for '{fname}' ---")

    # Retrieve the chunks AND embeddings for the file just processed
    new_chunks, new_embeddings = vector_manager.get_chunks_with_embeddings(fname)

    
    if (new_chunks and                                                  # Check if array exists and has elements
    new_embeddings is not None and 
    len(new_embeddings) > 0 and 
    any(len(emb) > 0 for emb in new_embeddings)):
        
        clusterizer = ChunkClusterizer(new_chunks, new_embeddings)      # Pass the chunks and their pre-computed embeddings to the clusterizer
        clusterizer.cluster_chunks()
    else:
        print(f"Could not retrieve chunks for {fname} to cluster.")    
    


def process_all_new():
    new_files = get_new_files()
    if not new_files:
        print("No new files to process.")
        return
    
    print(f"Found {len(new_files)} new files to process: {new_files}")
    
    for f in new_files:
        process_file(f)
    print("\nAll new files processed.")

### Option 1: Process all new files

In [ ]:
process_all_new()

Found 1 new files to process: ['note-2-ul.txt']

Processing new file: note-2-ul.txt

Chunk 1 (7 tokens):
unsup = no labels.

Chunk 2 (12 tokens):
kmeans = simplest one but still used a lot.

Chunk 3 (59 tokens):
init k centroids randomly (k-means++ better), assign pts, recalc centroids, repeat. converge when no pt changes. but result depends on init + scale. scale important! feature w bigger range dominates dist calc — always standardize first.

Chunk 4 (19 tokens):
elbow method = plot inertia vs k — look for bend, but not always obvious.

Chunk 5 (13 tokens):
inertia = sum of dist² to centroid.

Chunk 6 (20 tokens):
alt metric = silhouette score — between -1 and 1. close to 1 = well-clustered.

Chunk 7 (73 tokens):
clustering ≠ classification. labels are not known. use cases: market segmentation, gene expr clustering, anomaly detection (esp dbscan). DBSCAN better for weird shapes, dense clusters — uses eps + min_samples. tricky to tune tho. forms clusters based on density, noisy pts m

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Option 2: Process 1 file

In [ ]:
#process_file("note-1-dt-r.txt")

## Continue pipeline...
   5. Label each cluster based on existing labels in Chroma DB, if no match, create a new label
      1. maybe prompt to LLM to generate labels?
   6. Rewrite each labeled-cluster to connect all the chunks and create a coherent sub-document 
      1. use LLM for that.
      2. output should be like: sub-document 1 (label 1), sub-document 2 (label 2)
   7. Store each sub-document in correspondent label as a new file
      1. For test, generate a new folder locally and save the sub-document as .txt file in the folder with the correspondent label
      2. Use Notion API, the sub-document should be a new file in a Notion database, with the correpondent Label and metadata
   8. Add more documents in bulk and generate insights for simple EDA
       1.  Number of labels, sub-documents in each label, etc.

In [ ]:
# 5. Label clusters with classifier

In [ ]:
# 6. Rewrite selected outputs with [llama 3.2]

In [ ]:
#    7. Store each sub-document in correspondent label as a new file
#      1. For test, generate a new folder locally and save the sub-document as .txt file in the folder with the correspondent label


In [ ]:
# 8.    8. Add more documents in bulk and generate insights for simple EDA


# clean code

In [ ]:
# ### IMPORT
import warnings
warnings.filterwarnings('ignore')
import os
import re
import nltk
from transformers import AutoTokenizer
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
from IPython.display import Markdown, display
import hdbscan
import numpy as np
nltk.download('punkt')

# ### CONFIGURATION
FOLDER_PATH = "./data/mock_notes/"          # Input files directory
LOG_PATH = "./data/processed_files.txt"     # Processed files log
PERSIST_DIR = "./data/chroma_db"            # Vector database directory

# Create required directories
os.makedirs(FOLDER_PATH, exist_ok=True)
os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)
os.makedirs(PERSIST_DIR, exist_ok=True)

# Initialize embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# ### UTILITY FUNCTIONS
def get_processed_files():
    """Load list of already processed files from log."""
    if os.path.exists(LOG_PATH):
        with open(LOG_PATH, "r") as f:
            return set(line.strip() for line in f)
    return set()

def get_new_files():
    """Get list of new .txt files that haven't been processed yet."""
    if not os.path.exists(FOLDER_PATH):
        return []
    
    all_files = [f for f in os.listdir(FOLDER_PATH) if f.endswith(".txt")]
    processed_files = get_processed_files()
    return [f for f in all_files if f not in processed_files]

def count_tokens(text):
    """Count tokens in text using the embedding model's tokenizer."""
    return len(tokenizer.tokenize(text))

def is_code_like(sentence):
    """Detect if a sentence contains code-like patterns."""
    code_keywords = ["from ", "import ", "def ", "return ", "class ", "for ", "if ", "else", "while ", "try:", "except", "print("]
    code_symbols = ["=", "()", "[]", "{}", "->", "::"]
    sentence_lower = sentence.strip().lower()
    return (
        any(sentence_lower.startswith(k) for k in code_keywords) or
        any(sym in sentence for sym in code_symbols)
    )

# ### CHUNKING LOGIC
def chunk_text(text, source_name, min_tokens=40, max_tokens=100, inspect=False):
    """Split text into semantic chunks with token limits."""
    raw_blocks = re.split(r"\n\s*\n", text.strip())                    # Split by double newlines
    chunks, current_chunk, current_tokens = [], [], 0                  # Initialize chunking variables
    metadata = []                                                      # Store metadata for each chunk

    for block in raw_blocks:                                           # Process each text block
        sentences = nltk.sent_tokenize(block.strip())                  # Split block into sentences

        for sentence in sentences:                                     # Process each sentence
            sentence = sentence.strip()
            tokens = count_tokens(sentence)                            # Count tokens in sentence

            # Handle code-like sentences separately
            if is_code_like(sentence):                                 # Check if sentence contains code
                if current_chunk:                                      # Save current chunk if exists
                    chunks.append(" ".join(current_chunk))
                    metadata.append({"source": source_name})
                    current_chunk, current_tokens = [], 0
                chunks.append(sentence)                                # Add code as separate chunk
                metadata.append({"source": source_name})
                continue

            # Handle conversation transitions
            if any(sentence.lower().startswith(w) for w in ["btw", "wait", "oh", "then", "also", "now", "next"]):
                if current_chunk:                                      # Break chunk at transition words
                    chunks.append(" ".join(current_chunk))
                    metadata.append({"source": source_name})
                    current_chunk, current_tokens = [], 0

            # Check if adding sentence would exceed max tokens
            if current_tokens + tokens > max_tokens and current_tokens >= min_tokens:
                chunks.append(" ".join(current_chunk))                # Save current chunk
                metadata.append({"source": source_name})
                current_chunk, current_tokens = [], 0                 # Reset for new chunk

            current_chunk.append(sentence)                            # Add sentence to current chunk
            current_tokens += tokens                                  # Update token count

        # Save any remaining chunk after block processing
        if current_chunk:
            chunks.append(" ".join(current_chunk))
            metadata.append({"source": source_name})
            current_chunk, current_tokens = [], 0

    if inspect:                                                       # Optional: print chunks for debugging
        for i, chunk in enumerate(chunks):
            print(f"\nChunk {i+1} ({count_tokens(chunk)} tokens):\n{chunk}")

    return chunks, metadata

# ### VECTOR STORE MANAGEMENT
class VectorStoreManager:
    def __init__(self, persist_dir, embeddings):
        self.persist_dir = persist_dir                                # Store database directory path
        self.embeddings = embeddings                                  # Store embedding model
        self.vectorstore = self._load_existing()                     # Load existing DB or set to None

    def _load_existing(self):
        """Load existing Chroma database if it exists."""
        try:
            if os.path.exists(self.persist_dir) and os.listdir(self.persist_dir):
                print(f"Loading existing Chroma DB from {self.persist_dir}")
                return Chroma(
                    persist_directory=self.persist_dir,
                    embedding_function=self.embeddings,
                    collection_name="default"
                )
        except Exception as e:
            print(f"Error loading existing Chroma DB: {e}")
        
        print("No existing Chroma DB found. Will create new one.")
        return None

    def add_chunks(self, chunks, metadata):
        """Add text chunks to vector store with embeddings."""
        if not self.vectorstore:                                     # Create new database if none exists
            print(f"Creating new Chroma DB at {self.persist_dir}")
            self.vectorstore = Chroma.from_texts(
                texts=chunks,
                metadatas=metadata,
                embedding=self.embeddings,
                persist_directory=self.persist_dir,
                collection_name="default"
            )
        else:                                                        # Add to existing database
            self.vectorstore.add_texts(texts=chunks, metadatas=metadata)

    def get_chunks_with_embeddings(self, fname):
        """Retrieve chunks and embeddings for a specific file."""
        results = self.vectorstore.get(                              # Query database for file chunks
            where={"source": fname},
            include=["documents", "embeddings"]
        )
        
        chunks = results.get('documents', [])                        # Extract text chunks
        embeddings_list = results.get('embeddings', [])             # Extract embeddings
        
        # Fallback: compute embeddings if not stored
        if not embeddings_list:                                      # Generate embeddings if missing
            print(f"Computing embeddings for {fname}...")
            embeddings_list = [self.embeddings.embed_query(chunk) for chunk in chunks]
        
        return chunks, embeddings_list

# ### CLUSTERING LOGIC
class ChunkClusterizer:
    def __init__(self, chunks, chunk_embeddings):
        self.chunks = chunks                                         # Store text chunks
        self.embeddings = np.array(chunk_embeddings)                # Convert embeddings to numpy array
        self.labels = []                                             # Initialize cluster labels

    def cluster_chunks(self, min_cluster_size=2, min_samples=1):
        """Cluster chunks using HDBSCAN algorithm."""
        print("Clustering chunks with HDBSCAN...")
        
        # Handle edge cases
        if len(self.chunks) < min_cluster_size:                      # Not enough chunks to cluster
            print(f"Not enough chunks to cluster ({len(self.chunks)} < {min_cluster_size})")
            self.labels = [-1] * len(self.chunks)                   # Assign all to noise
            return self.labels

        # Ensure embeddings are 2D
        if self.embeddings.ndim == 1:                               # Reshape 1D embeddings to 2D
            self.embeddings = self.embeddings.reshape(1, -1)

        print(f"Clustering {len(self.chunks)} chunks with embeddings shape: {self.embeddings.shape}")
        
        # Perform clustering
        clusterizer = hdbscan.HDBSCAN(                              # Initialize clustering algorithm
            min_cluster_size=min_cluster_size,
            min_samples=min_samples,
            metric='euclidean'
        )
        self.labels = clusterizer.fit_predict(self.embeddings)      # Generate cluster labels

        # Display results
        num_clusters = len(set(self.labels)) - (1 if -1 in self.labels else 0)  # Count clusters (exclude noise)
        print(f"Found {num_clusters} clusters")
        print(f"Cluster distribution: {dict(zip(*np.unique(self.labels, return_counts=True)))}")

        # Show cluster contents
        for cluster_id in sorted(set(self.labels)):                 # Iterate through each cluster
            cluster_name = "Noise" if cluster_id == -1 else f"Cluster {cluster_id}"
            chunk_count = sum(1 for label in self.labels if label == cluster_id)
            print(f"\n--- {cluster_name} ({chunk_count} chunks) ---")
            
            for i, label in enumerate(self.labels):                 # Show chunks in this cluster
                if label == cluster_id:
                    preview = self.chunks[i][:200] + ('...' if len(self.chunks[i]) > 200 else '')
                    print(f"Chunk {i+1}: {preview}\n")

        return self.labels

# ### MAIN PROCESSING PIPELINE
def process_file(fname):
    """Process a single file through the complete pipeline."""
    print(f"\n=== Processing: {fname} ===")
    
    # Step 1: Read file
    with open(os.path.join(FOLDER_PATH, fname), "r", encoding="utf-8") as f:
        text = f.read()

    # Step 2: Create chunks
    chunks, metadata = chunk_text(text, fname, inspect=True)        # Split text into chunks
    print(f"Created {len(chunks)} chunks")

    # Step 3: Store in vector database
    vector_manager = VectorStoreManager(PERSIST_DIR, embeddings)    # Initialize vector store
    vector_manager.add_chunks(chunks, metadata)                     # Add chunks with embeddings
    print(f"Stored {len(chunks)} chunks in vector database")

    # Step 4: Mark as processed
    with open(LOG_PATH, "a") as log:                                # Log processed file
        log.write(fname + "\n")

    # Step 5: Cluster chunks
    print(f"\n--- Clustering chunks from {fname} ---")
    new_chunks, new_embeddings = vector_manager.get_chunks_with_embeddings(fname)  # Retrieve stored data
    
    clusterizer = ChunkClusterizer(new_chunks, new_embeddings)      # Initialize clustering
    clusterizer.cluster_chunks()                                    # Perform clustering

def process_all_new():
    """Process all new files in the input directory."""
    new_files = get_new_files()                                     # Get list of unprocessed files
    
    if not new_files:                                               # Exit early if no new files
        print("No new files to process.")
        return
    
    print(f"Found {len(new_files)} new files: {new_files}")
    
    for fname in new_files:                                         # Process each new file
        process_file(fname)
    
    print(f"\n=== Completed processing {len(new_files)} files ===")

# ### EXECUTION
if __name__ == "__main__":
    process_all_new()                                               # Run the complete pipeline